In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [ ]:
def goTo(driver, query):
    script = f"document.querySelector(\'{query}\').click()"
    driver.execute_script(script)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "mainForm")))

def dump_html(driver, index):
    html_content = driver.page_source

    # Write to file with UTF-8 encoding (handles French characters)
    with open(f'epicea_results/{index}.html', 'w', encoding='utf-8') as f:
        f.write(html_content)

def scrape():
    # Setup Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Remove for visible browser
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # Launch browser
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        driver.get('https://epicea.inrs.fr/servlet/public_request')
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "mainForm")))

        driver.execute_script("window.mainForm.calculate.value = 'true';")
        driver.execute_script("window.mainForm.searchType.value = 'simple';")
        driver.execute_script("window.mainForm.submit();")

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "mainForm")))

        driver.execute_script("window.mainForm.goTo.value='1'")
        driver.execute_script("window.mainForm.submit();")

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "mainForm")))

        goTo(driver, 'a.lien[href*=\"public_display\"]')
        dump_html(driver, "00")
        start = time.time()
        for i in range(100):
            goTo(driver, 'a.lien[title=\"Dossier suivant\"]')
            dump_html(driver, str(i))
            #time.sleep(.1)
        print(f"Elapsed time : {time.time() - start}s")
        
    finally:
        driver.quit()

scrape()

Elapsed time : 4.474967956542969s


In [3]:
import os
import glob
from bs4 import BeautifulSoup
from pathlib import Path
from tqdm.notebook import tqdm

In [4]:
def process_html_directory(directory_path="./epicea_results", extension="*.html"):
    soups = {}
    for file_path in tqdm(glob.glob(os.path.join(directory_path, extension)), desc="Processing HTML files"):
        filename = os.path.basename(file_path)
        with open(file_path, 'r', encoding='utf-8') as f: content = f.read()
        soups[filename] = BeautifulSoup(content, 'html.parser')
    return soups

soups = process_html_directory()

Processing HTML files:   0%|          | 0/101 [00:00<?, ?it/s]

In [8]:
data = []
currentDossier = 0

for soup in tqdm(soups.values()):
    table = soup.select("table.tablein")[2]
    tmpDict = {}
    for tr in table.select("tr")[1:]:
        tds = tr.select("td")[0:2]
        title : str = tds[0].getText().strip().replace('\xa0', ' ').replace(' :', '')
        content : str = tds[1].getText().strip()
        tmpDict[title] = content
    data.append(tmpDict)


  0%|          | 0/101 [00:00<?, ?it/s]

In [6]:
import pandas as pd

In [7]:
x = pd.DataFrame(data)
x

,Numéro du dossier,Comité technique national,Code entreprise,Matériel en cause,Résumé de l'accident
0,27695,I - Activités de service II et travail temporaire,8790A - Hébergement social pour enfants en dif...,998000 - Sans objet,Une stagiaire éducatrice spécialisée de 19 ans...
1,27696,I - Activités de service II et travail temporaire,7820Z - Activités des agences de travail tempo...,050903 - Autre camion,"Vers 16h30, un conducteur de camion de 71 ans ..."
2,27694,B - Bâtiment et Travaux Publics,4391B - Travaux de couverture par éléments,510210 - Toiture en matériaux fragiles (verriè...,"Un couvreur de 43 ans, sous-traitant 1 de l’en..."
3,27685,H - Activités de service I,6419Z - Autres intermédiations monétaires,998000 - Sans objet,"Un chargé d'affaires de 47 ans, travaillant po..."
4,27684,"C - Transports, eau, gaz, électricité, livre e...",4941A - Transports routiers de fret interurbains,"051402 - Voiture, voiturette",Après avoir pris son café dans un relais routi...
...,...,...,...,...,...
96,27600,B - Bâtiment et Travaux Publics,4339Z - Autres travaux de finition,"051302 - Véhicules utilitaires légers, camionn...",Un manoeuvre de 36 ans se rendait chez un clie...
97,27599,"C - Transports, eau, gaz, électricité, livre e...",4941B - Transports routiers de fret de proximité,998000 - Sans objet,Un conducteur de 60 ans devait prendre son cam...
98,27598,"D - Services, commerces et industries de l'ali...",5510Z - Hôtels et hébergement similaire,050909 - Véhicule sur route non précisé,Une serveuse de 20 ans a eu un accident de la ...
99,27597,H - Activités de service I,8412Z - Administration publique (tutelle) de l...,"051401 - Cycle à deux (ou trois) roues, (motor...",Une accompagnante d'élève en situation de hand...
